In [ ]:
import numpy as np
import inference,hpwren
import tflite_runtime.interpreter as tflite
import time,datetime,os,sys,subprocess
from distutils.util import strtobool
import json

## Input HPWREN Site ID and Camera type

In [ ]:
cameraType : 'CWLIntInput' = 0
siteID     : 'CWLIntInput' = 0

object = 'model.tflite'
modelDir = 'src/'
modelPath : 'CWLFilePathInput' = os.path.abspath(os.path.join(modelDir,object))

## Output files for smoke detection model

In [ ]:
resultsDir = 'src/'
imageName = 'hpwren-image-used-for-inference.jpeg'
resultsName = 'model-inference-results.json'

imagePath  : 'CWLFilePathOutput'  = imageName
resultsPath: 'CWLFilePathOutput'  = resultsName

## Perform an inference on selected hpwren site and camera type

In [ ]:
#HPWREN Parameters
hpwrenUrl = "https://firemap.sdsc.edu/pylaski/"\
"stations?camera=only&selection="\
"boundingBox&minLat=0&maxLat=90&minLon=-180&maxLon=0"
camObj = hpwren.cameras(hpwrenUrl)
serverName = 'HPWREN Camera'
imageURL,description = camObj.getImageURL(cameraType,siteID)

#Inference Section
print('Starting smoke detection inferencing')
testObj = inference.FireImage()
print('Get image from ' + serverName)
print("Image url: " + imageURL)
print("Description: " + description)
testObj.urlToImage(imageURL)
testObj.writeImage(imagePath)

interpreter = tflite.Interpreter(model_path=modelPath)
interpreter.allocate_tensors()
result,percent = testObj.inference(interpreter)

## Output results

In [ ]:
print('Perform an inference based on trainned model')
print(result)

#Output section
classifier = result.split(',')[0]
currentDT = str(datetime.datetime.now())
outputDataSchema = ['Image Server','Image URL','Image Description',
                    'Inference Classifier','Inference Accuracy',
                    'Inference Datetime']
outputDataVals = [serverName,imageURL,description,
                    classifier,percent,currentDT]
outputDataDict = {outputDataSchema[i]: outputDataVals[i] for i in range(len(outputDataSchema))}
with open(resultsPath,'w') as fp:
    json.dump(outputDataDict,fp)